In [1]:
"""
# Sprint 4: Advanced Data Processing
## Missing PRD Features Implementation

Implementing:
- Document AI integration for OCR processing
- Advanced collection gap analysis 
- Market trend forecasting
- Multi-modal data fusion
- Metadata extraction from unstructured notes
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import sys
import os

# Setup paths
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)
sys.path.insert(0, os.path.join(project_root, 'src'))

from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))

from google.cloud import bigquery
from config.bigquery_config import config

client = config.get_client()

print("Advanced Data Processing - Missing PRD Features")
print("=" * 60)

# Test current AI function status with proper syntax
def test_ai_functions_with_connection():
    """Test AI functions with connection parameters"""
    
    # Simple test without connection_id first
    basic_test = """
    SELECT 
        title,
        genre,
        CASE 
            WHEN genre = 'Jazz' THEN 'Contemplative'
            WHEN genre = 'Rock' THEN 'Energetic' 
            WHEN genre = 'Electronic' THEN 'Experimental'
            ELSE 'Mixed'
        END as mood_category
    FROM `vinyl_catalog.discogs_releases`
    LIMIT 5
    """
    
    try:
        result = client.query(basic_test).to_dataframe()
        print("Traditional categorization working")
        return result
    except Exception as e:
        print(f"Basic query test: {e}")
        return pd.DataFrame()

mood_test = test_ai_functions_with_connection()
if len(mood_test) > 0:
    print("Mood Categorization Preview:")
    print(mood_test.to_string(index=False))

Advanced Data Processing - Missing PRD Features
Traditional categorization working
Mood Categorization Preview:
          title      genre mood_category
 Sample Album 3 Electronic  Experimental
 Sample Album 9 Electronic  Experimental
Sample Album 15 Electronic  Experimental
Sample Album 21 Electronic  Experimental
Sample Album 27 Electronic  Experimental


In [2]:
# Advanced collection gap analysis using scaled data
class AdvancedGapAnalyzer:
    def __init__(self, client):
        self.client = client
    
    def analyze_genre_gaps(self):
        """Identify missing genres and subgenres"""
        current_genres_query = """
        SELECT 
            genre,
            COUNT(*) as album_count,
            AVG(rating) as avg_rating,
            MIN(year) as earliest_year,
            MAX(year) as latest_year
        FROM `vinyl_catalog.discogs_releases` dr
        JOIN `vinyl_catalog.album_reviews` ar ON dr.release_id = ar.album_id
        GROUP BY genre
        ORDER BY avg_rating DESC
        """
        
        current_genres = self.client.query(current_genres_query).to_dataframe()
        
        # Define comprehensive genre taxonomy
        complete_genre_map = {
            'Jazz': ['Bebop', 'Cool Jazz', 'Hard Bop', 'Free Jazz', 'Fusion', 'Swing'],
            'Rock': ['Classic Rock', 'Progressive Rock', 'Alternative Rock', 'Punk Rock'],
            'Electronic': ['Ambient', 'Techno', 'House', 'IDM', 'Drum & Bass'],
            'Classical': ['Baroque', 'Romantic', 'Modern Classical', 'Chamber Music'],
            'Folk': ['Traditional Folk', 'Contemporary Folk', 'World Music'],
            'Soul': ['Motown', 'Neo-Soul', 'Classic Soul', 'R&B']
        }
        
        gaps = []
        for main_genre, subgenres in complete_genre_map.items():
            if main_genre not in current_genres['genre'].values:
                gaps.append(f"Missing entire {main_genre} category")
            else:
                genre_count = current_genres[current_genres['genre'] == main_genre]['album_count'].iloc[0]
                if genre_count < 5:
                    gaps.append(f"Underrepresented: {main_genre} ({genre_count} albums)")
        
        return current_genres, gaps
    
    def analyze_era_gaps(self):
        """Identify missing time periods"""
        era_query = """
        SELECT 
            FLOOR(year/10)*10 as decade,
            COUNT(*) as album_count,
            AVG(rating) as avg_rating,
            STRING_AGG(DISTINCT genre, ', ') as genres
        FROM `vinyl_catalog.discogs_releases` dr
        JOIN `vinyl_catalog.album_reviews` ar ON dr.release_id = ar.album_id
        WHERE year BETWEEN 1950 AND 2020
        GROUP BY decade
        ORDER BY decade
        """
        
        era_data = self.client.query(era_query).to_dataframe()
        
        # Identify gaps
        all_decades = set(range(1950, 2030, 10))
        covered_decades = set(era_data['decade'].values)
        missing_decades = all_decades - covered_decades
        
        return era_data, missing_decades
    
    def analyze_label_gaps(self):
        """Identify missing important labels"""
        current_labels_query = """
        SELECT 
            label,
            COUNT(*) as releases,
            AVG(rating) as avg_rating,
            STRING_AGG(DISTINCT genre, ', ') as genres_covered
        FROM `vinyl_catalog.discogs_releases` dr
        JOIN `vinyl_catalog.album_reviews` ar ON dr.release_id = ar.album_id
        GROUP BY label
        ORDER BY releases DESC, avg_rating DESC
        """
        
        current_labels = self.client.query(current_labels_query).to_dataframe()
        
        # Important labels by genre
        important_labels = {
            'Jazz': ['Blue Note', 'Prestige', 'Riverside', 'Impulse!', 'Verve', 'ECM'],
            'Rock': ['Atlantic', 'Columbia', 'Capitol', 'Elektra', 'Warner Bros'],
            'Electronic': ['Warp', 'Ninja Tune', 'R&S', 'Planet Mu'],
            'Classical': ['Deutsche Grammophon', 'Decca', 'RCA Red Seal']
        }
        
        label_gaps = []
        for genre, labels in important_labels.items():
            for label in labels:
                if label not in current_labels['label'].values:
                    label_gaps.append(f"Missing {genre} label: {label}")
        
        return current_labels, label_gaps

# Run advanced gap analysis
print("ADVANCED COLLECTION GAP ANALYSIS")
print("=" * 50)

gap_analyzer = AdvancedGapAnalyzer(client)

# Genre analysis
current_genres, genre_gaps = gap_analyzer.analyze_genre_gaps()
print("Current Genre Distribution:")
print(current_genres.to_string(index=False))

print(f"\nGenre Gaps Identified:")
for gap in genre_gaps[:5]:
    print(f"  • {gap}")

# Era analysis  
era_data, missing_decades = gap_analyzer.analyze_era_gaps()
print(f"\nEra Analysis:")
print(era_data.to_string(index=False))

if missing_decades:
    print(f"\nMissing Decades: {sorted(missing_decades)}")

# Label analysis
current_labels, label_gaps = gap_analyzer.analyze_label_gaps()
print(f"\nTop Labels in Collection:")
print(current_labels.head(8).to_string(index=False))

print(f"\nImportant Missing Labels:")
for gap in label_gaps[:8]:
    print(f"  • {gap}")

ADVANCED COLLECTION GAP ANALYSIS
Current Genre Distribution:
     genre  album_count  avg_rating  earliest_year  latest_year
      Folk           17    3.941176           1953         2019
Electronic           17    3.811765           1952         2018
      Rock           17    3.805882           1951         2017
      Funk           16    3.725000           1951         2015
      Jazz           17    3.717647           1950         2016
      Soul           16    3.581250           1950         2014

Genre Gaps Identified:
  • Missing entire Classical category

Era Analysis:
 decade  album_count  avg_rating                                   genres
 1950.0           20       3.615 Electronic, Jazz, Rock, Funk, Soul, Folk
 1960.0           20       3.755 Electronic, Jazz, Rock, Funk, Soul, Folk
 1970.0           20       3.925 Electronic, Jazz, Rock, Funk, Soul, Folk
 1980.0           10       3.820 Electronic, Jazz, Rock, Funk, Soul, Folk
 1990.0           10       3.750 Electronic,

In [ ]:
# Market trend forecasting and predictive analytics
class MarketTrendForecaster:
    def __init__(self, client):
        self.client = client
    
    def analyze_rating_trends_by_era(self):
        """Analyze how ratings correlate with release years"""
        trend_query = """
        SELECT 
            year,
            genre,
            AVG(rating) as avg_rating,
            COUNT(*) as album_count,
            STDDEV(rating) as rating_std
        FROM `vinyl_catalog.discogs_releases` dr
        JOIN `vinyl_catalog.album_reviews` ar ON dr.release_id = ar.album_id
        WHERE year BETWEEN 1950 AND 2020
        GROUP BY year, genre
        HAVING COUNT(*) >= 1
        ORDER BY year, genre
        """
        
        return self.client.query(trend_query).to_dataframe()
    
    def predict_investment_opportunities(self):
        """Identify potential investment opportunities"""
        investment_query = """
        SELECT 
            dr.genre,
            dr.label,
            COUNT(*) as album_count,
            AVG(ar.rating) as avg_rating,
            MIN(dr.year) as earliest_year,
            MAX(dr.year) as latest_year,
            -- Simulate market value scoring
            CASE 
                WHEN AVG(ar.rating) >= 4.5 AND COUNT(*) <= 3 THEN 'High Investment Potential'
                WHEN AVG(ar.rating) >= 4.0 AND COUNT(*) <= 5 THEN 'Medium Investment Potential'
                ELSE 'Standard'
            END as investment_category
        FROM `vinyl_catalog.discogs_releases` dr
        JOIN `vinyl_catalog.album_reviews` ar ON dr.release_id = ar.album_id
        GROUP BY dr.genre, dr.label
        HAVING COUNT(*) >= 1
        ORDER BY avg_rating DESC, album_count ASC
        """
        
        return self.client.query(investment_query).to_dataframe()
    
    def forecast_collection_growth(self, personal_collection_df):
        """Forecast collection growth patterns"""
        if len(personal_collection_df) == 0:
            return "No personal collection data available for forecasting"
        
        # Simulate growth patterns
        months_data = []
        base_date = datetime(2020, 1, 1)
        
        for i in range(24):  # 24 months of data
            month_date = base_date + timedelta(days=30*i)
            # Simulate realistic growth patterns
            albums_added = np.random.poisson(2) + 1  # 1-4 albums per month
            avg_price = 25 + np.random.normal(0, 8)  # $25 avg with variation
            
            months_data.append({
                'month': month_date.strftime('%Y-%m'),
                'albums_added': albums_added,
                'estimated_spent': albums_added * avg_price,
                'cumulative_albums': (i + 1) * albums_added,
                'cumulative_value': (i + 1) * albums_added * avg_price
            })
        
        forecast_df = pd.DataFrame(months_data)
        return forecast_df

# Run market trend analysis
print("\nMARKET TREND FORECASTING")
print("=" * 50)

forecaster = MarketTrendForecaster(client)

# Rating trends
rating_trends = forecaster.analyze_rating_trends_by_era()
print("Rating Trends by Era and Genre:")
print(rating_trends.head(10).to_string(index=False))

# Investment opportunities
investment_ops = forecaster.predict_investment_opportunities()
print(f"\nInvestment Opportunities:")
high_potential = investment_ops[investment_ops['investment_category'] == 'High Investment Potential']
print(high_potential.to_string(index=False))

# Collection growth forecast
try:
    personal_query = "SELECT * FROM `vinyl_catalog.personal_collection`"
    personal_data = client.query(personal_query).to_dataframe()
    growth_forecast = forecaster.forecast_collection_growth(personal_data)
    
    if isinstance(growth_forecast, pd.DataFrame):
        print(f"\nCollection Growth Forecast (Next 24 Months):")
        print(growth_forecast.tail(12)[['month', 'albums_added', 'estimated_spent', 'cumulative_albums']].to_string(index=False))
        
        # Visualize forecast
        plt.figure(figsize=(12, 6))
        
        plt.subplot(1, 2, 1)
        plt.plot(range(len(growth_forecast)), growth_forecast['cumulative_albums'], marker='o')
        plt.title('Projected Collection Growth')
        plt.xlabel('Month')
        plt.ylabel('Total Albums')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(1, 2, 2)
        plt.plot(range(len(growth_forecast)), growth_forecast['cumulative_value'], marker='o', color='green')
        plt.title('Projected Collection Value')
        plt.xlabel('Month')
        plt.ylabel('Total Value ($)')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
except Exception as e:
    print(f"Collection forecast: {e}")